In [22]:
import time 
import requests, bs4
import re
import sys
import os
import datetime
from urllib.parse import urlparse

import pandas as pd
import numpy as np
from omegaconf import OmegaConf

sys.path.append("../")
from src.scraping import ScrapingBase, ScrapingSponavi

%matplotlib inline
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 出力先
conf_dir = "../config"
conf_cli = OmegaConf.from_cli()
conf_exec = OmegaConf.load(os.path.join(conf_dir, "config_exec.yaml"))
conf_path = OmegaConf.load(os.path.join(conf_dir, "config_path.yaml"))
conf_url = OmegaConf.load(os.path.join(conf_dir, "config_url.yaml"))
conf_team = OmegaConf.load(os.path.join(conf_dir, "config_team.yaml"))
config = OmegaConf.merge(conf_cli, conf_exec, conf_path, conf_url, conf_team)

[autoreload of src.scraping failed: Traceback (most recent call last):
  File "/mnt/c/Users/kosuke/project/ikkyu_sokuho_data/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/mnt/c/Users/kosuke/project/ikkyu_sokuho_data/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/mnt/c/Users/kosuke/tool/anaconda3/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/mnt/c/Users/kosuke/tool/anaconda3/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._

In [49]:
url = "https://baseball.yahoo.co.jp/npb/player/1700049/top"
html = ss.get_html(url)
soup = bs4.BeautifulSoup(html, "html.parser")

get html  https://baseball.yahoo.co.jp/npb/player/1700049/top
success


In [9]:
result = {}
soup_name = soup.select_one("ruby[class='bb-profile__name']")
result["player_name"] = soup_name.select_one("h1").get_text()
result["player_name_kana"] = soup_name.select_one("rt").get_text()[1:-1]
result["number"] = soup.select_one("p.bb-profile__number").get_text()

list_text = [x.get_text() for x in soup.select("dd[class='bb-profile__text']")]
result["birth_day"] = list_text[0]
result["birth_place"] = list_text[1]
result["height"] = list_text[2]
result["weight"] = list_text[3]
result["blood_type"] = list_text[4]
result["bating_hand"] = list_text[5]
result["draft_year"] = list_text[6]
result["pro_age"] = list_text[7]
result["keireki"] = list_text[8]
result["profile_text"] = soup.select_one("p[class='bb-profile__summary']").get_text()
result["img_url"] = soup.select_one("div[class='bb-profile__photo']").select_one("img").get("src")

result

AttributeError: 'NoneType' object has no attribute 'select_one'

In [66]:
ss.teamtype

AttributeError: 'ScrapingSponavi' object has no attribute 'teamtype'

In [16]:
players_url = ss.base_url + "/teams/" + team_info.team_id.replace("npb", "") + f"/memberlist?kind={kind}"

df_player_info = pd.DataFrame()
# チームの選手ページをループ
for team in ss.team_list[:1]:
    for kind in ["p", "b"]:
        team_info = ss.team_dict[team]
        print(players_url)
        players_url = ss.base_url + "/teams/" + team_info.team_id.replace("npb", "") + f"/memberlist?kind={kind}"
        players_html = ss.get_html(players_url)
        soup = bs4.BeautifulSoup(players_html, "html.parser")
        list_players_pre = soup.select("td[class='bb-playerTable__data bb-playerTable__data--player']")
        list_players = [x.select_one("a").get("href").split("/")[-2] for x in list_players_pre]

        # 順に選手情報を取得
        for player_num in list_players:
            player_url = f"https://baseball.yahoo.co.jp/npb/player/{player_num}/top"
            player_html = ss.get_html(player_url)
            result = ss.get_player_info(player_html)
            result["player_id"] = ss.make_id(player_num)

            # dfを結合
            df_player_info = df_player_info.append(pd.Series(result), ignore_index=True)

df_player_info["exec_datetime"] = "0"
df_player_info = df_player_info[ss.columns.lake_player]
print(df_player_info)

 'number': '45'}
1994年5月31日（26歳）
広島
186cm
82kg
AB
右投げ左打ち
2016年（2位）
5年
近大福山高－近畿大－巨人
0
1
2
3
4
5
6
7
8
get html  https://baseball.yahoo.co.jp/npb/player/1800023/top
success
{'player_name': '髙橋 優貴', 'player_name_kana': 'タカハシ ユウキ', 'number': '47'}
1997年2月1日（24歳）
茨城
178cm
82kg
B
左投げ左打ち
2018年（1位）
3年
東海大菅生高－八戸学院大－巨人
0
1
2
3
4
5
6
7
8
get html  https://baseball.yahoo.co.jp/npb/player/1900136/top
success
{'player_name': 'ビエイラ', 'player_name_kana': nan, 'number': '49'}
1993年1月7日（28歳）
ブラジル
193cm
113kg
不明
右投げ右打ち
2年
カントリーキッズ高－マリナーズ－ホワイトソックス－巨人
0
1
2
3
4
5
6
7
get html  https://baseball.yahoo.co.jp/npb/player/1400102/top
success
{'player_name': '戸根 千明', 'player_name_kana': 'トネ チアキ', 'number': '50'}
1992年10月17日（28歳）
京都
174cm
100kg
A
左投げ左打ち
2014年（2位）
7年
石見智翠館高－日本大－巨人
0
1
2
3
4
5
6
7
8
get html  https://baseball.yahoo.co.jp/npb/player/1600100/top
success
{'player_name': '高梨 雄平', 'player_name_kana': 'タカナシ ユウヘイ', 'number': '53'}
1992年7月13日（28歳）
埼玉
175cm
81kg
O
左投げ左打ち
2016年（9位）
5年
川越東高－早稲田大－JX-ENEOS－楽天－巨人

AttributeError: 'ScrapingSponavi' object has no attribute 'columns'

In [23]:
print(ss.column["lake_player"])
# df_player_info

TypeError: 'NoneType' object is not subscriptable

In [29]:
ss.column["lake_player"]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
ss.save_csv(df_all, file_path="../data/player_result/lake_player_result")

In [31]:
ss.column

In [32]:
soup = bs4.BeautifulSoup(html, "html.parser")
tb = soup.find_all('table')[0] 
df = pd.read_html(str(tb),encoding='utf-8', header=0)[0]
df = df.drop(len(df)-1) # 最終行を削除
df['player_id'] = [ss.get_id(row.select_one("a").get("href").split("/")[-2]) for row in tb.select("tr[class='bb-playerTable__row']")]
df[df == "-"] = np.nan

,背番号,選手名,防御率,登 板,先 発,完 投,完 封,Ｑ Ｓ,勝 利,敗 戦,...,与四球,与死球,暴 投,ボーク,失 点,自責点,被打率,Ｋ／ＢＢ,ＷＨＩＰ,player_id
0,11,平内 龍太,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb2000005
1,12,デラロサ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1900012
2,15,サンチェス,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1900143
3,17,大竹 寛,0.00,1,0,0,0,0,0,0,...,0,0,0,0,0,0,.000,NaN,0.00,npb12179
4,18,菅野 智之,4.50,1,1,0,0,1,0,0,...,3,0,0,0,3,3,.333,1.33,1.83,npb1200041
5,19,山崎 伊織,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb2000006
6,20,戸郷 翔征,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1800028
7,21,井納 翔一,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1200050
8,23,野上 亮磨,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb900055
9,26,今村 信貴,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,npb1100038


,背番号,選手名,防御率,登 板,先 発,完 投,完 封,Ｑ Ｓ,勝 利,敗 戦,...,奪三振率,与四球,与死球,暴 投,ボーク,失 点,自責点,被打率,Ｋ／ＢＢ,ＷＨＩＰ
0,11,平内 龍太,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,12,デラロサ,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,15,サンチェス,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,17,大竹 寛,0.00,1,0,0,0,0,0,0,...,0.00,0,0,0,0,0,0,.000,-,0.00
4,18,菅野 智之,4.50,1,1,0,0,1,0,0,...,6.00,3,0,0,0,3,3,.333,1.33,1.83
5,19,山崎 伊織,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,20,戸郷 翔征,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,21,井納 翔一,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,23,野上 亮磨,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,26,今村 信貴,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


['npb2000005',
 'npb1900012',
 'npb1900143',
 'npb12179',
 'npb1200041',
 'npb2000006',
 'npb1800028',
 'npb1200050',
 'npb900055',
 'npb1100038',
 'npb1200089',
 'npb1900017',
 'npb1500046',
 'npb1500052',
 'npb1700002',
 'npb1600060',
 'npb1800023',
 'npb1900136',
 'npb1400102',
 'npb1600100',
 'npb2000008',
 'npb1100040',
 'npb2000010',
 'npb1500113',
 'npb1800026',
 'npb1300087',
 'npb1600064',
 'npb1900019',
 'npb1800031',
 'npb1600072',
 'npb1600068',
 'npb2000014',
 'npb1800030',
 'npb2000020',
 'npb2000015',
 'npb1700053',
 'npb2000018',
 'npb1700042',
 'npb2000021',
 'npb1900016',
 'npb1600061',
 'npb1500048',
 'npb2000019',
 'npb1800025']

In [11]:
96-45 

51

In [5]:
dfs

[    èçªå·         é¸æå  é²å¾¡ç  ç»ãæ¿  å
 ãçº  å®ãæ  \
 0          11      å¹³å é¾å¤ª          -          -          -          -   
 1          12      ãã©ã­ãµ          -          -          -          -   
 2          15   ãµã³ãã§ã¹          -          -          -          -   
 3          17        å¤§ç«¹ å¯       0.00          1          0          0   
 4          18     è
 é æºä¹       4.50          1          1          0   
 5          19     å±±å´ ä¼ç¹          -          -          -          -   
 6          20     æ¸é· ç¿å¾          -          -          -          -   
 7          21     äºç´ ç¿ä¸          -          -          -          -   
 8          23     éä¸ äº®ç£¨          -          -          -          -   
 9          26     ä»æ ä¿¡è²´          -          -          -          -   
 10         30     éµè°· é½å¹³       0.00          1          0          0   
 11         33        å¤ªç° é¾      

In [ ]:
url_schedule_day = 'https://baseball.yahoo.co.jp/npb/schedule/

In [20]:
import re

content = "https://baseball.yahoo.co.jp/npb/game/2021000011/index"
result = re.search("\d+", content)
if result:
    print(result.group())

2021000011


In [8]:
print(url)
index = re.search(r'index=\d+', url)
res = requests.get(url)

try:
    res.raise_for_status() 
except requests.exceptions.RequestException:
    print('Not Found Error in {}'.format(index.group())) #Not Foundエラーが出たらその回は終わったとみなす
    #break

soup = bs4.BeautifulSoup(res.text, "html.parser")
elems = soup.select('.bb-calendarTable__status')
elems2 = soup.select('.bb-calendarTable__venue')

#print(elems)
for elem,elem2 in zip(elems,elems2):
    url = elem.get('href')
    date = url[38:46]
    venue = elem2.text
    #print(venue)
    game_yokohama.loc[len(game_yokohama)] = [url,date,venue]
time.sleep(1)

NameError: name 'url' is not defined